In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train_data=pd.read_csv("../input/train.csv")
test_data=pd.read_csv("../input/test.csv")

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data.Cover_Type.value_counts()

In [ ]:
real_data_columns=["Elevation","Aspect","Slope","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",
          "Hillshade_9am","Hillshade_Noon","Hillshade_3pm","Horizontal_Distance_To_Fire_Points","Horizontal_Distance_To_Roadways"]

In [ ]:
sns.pairplot(train_data[real_data_columns+["Cover_Type"]],hue="Cover_Type")

In [ ]:
train_data.Soil_Type40.value_counts()

In [ ]:
train_data=train_data.drop(['Soil_Type25'],axis=1)
test_data=test_data.drop(['Soil_Type25'],axis=1)

In [ ]:
train_data=train_data.drop(['Soil_Type7'],axis=1)
test_data=test_data.drop(['Soil_Type7'],axis=1)

In [ ]:
print(train_data.shape)
print(test_data.shape)

In [ ]:
test_id=test_data["Id"].values
test_data=test_data.drop(["Id"],axis=1)
train_target=train_data["Cover_Type"].values
train_data=train_data.drop(["Id","Cover_Type"],axis=1)

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data_real=train_data[real_data_columns]
test_data_real=test_data[real_data_columns]
train_data_bynary=train_data.drop(real_data_columns,axis=1)
test_data_bynary=test_data.drop(real_data_columns,axis=1)
mean=train_data_real.mean(axis=0)
std=train_data_real.std(axis=0)
train_data_real-=mean
train_data_real/=std
test_data_real-=mean
test_data_real/=std
X_train=np.hstack((train_data_real,train_data_bynary))
X_test=np.hstack((test_data_real,test_data_bynary))
mapping={1:0,2:1,3:2,4:3,5:4,6:5,7:6}
Y_train=[mapping[y] for y in train_target]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
training_data,val_data,training_target,val_target=train_test_split(X_train,
                                                                   Y_train,test_size=0.3)

In [ ]:
from keras import models
from keras import layers
from keras import regularizers

In [ ]:
model=models.Sequential()
model.add(layers.Dense(1024,kernel_regularizer=regularizers.l2(0.001),activation="relu",
                       input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.001),activation="relu"))
model.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.001),activation="relu"))
model.add(layers.Dense(64,activation="relu"))
model.add(layers.Dense(7,activation='softmax'))
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(training_data,training_target,epochs=45,batch_size=128,validation_data=(val_data,val_target))

In [ ]:
acc=history.history['acc']
val_acc=history.history['val_acc']
epochs = range(1, len(val_acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training loss')
plt.plot(epochs, val_acc, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

In [ ]:
model=models.Sequential()
model.add(layers.Dense(1024,kernel_regularizer=regularizers.l2(0.001),activation="relu",
                       input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.001),activation="relu"))
model.add(layers.Dense(256,kernel_regularizer=regularizers.l2(0.001),activation="relu"))
model.add(layers.Dense(7,activation='softmax'))
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train,epochs=45,batch_size=128)

In [ ]:
predictions=model.predict(X_test)
predictions=np.argmax(predictions,axis=1)
mapping={0:1,1:2,2:3,3:4,4:5,5:6,6:7}
predictions=[mapping[y] for y in predictions]
data_submission=pd.DataFrame()
data_submission['Id']=test_id
data_submission["Cover_Type"]=predictions
data_submission.to_csv("my_submission.csv",index=False)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
%%time
cv=5
clf=XGBClassifier(learning_rate=0.775,n_estimators=512,max_depth=16)
scores=cross_val_score(clf,training_data,training_target,scoring="accuracy",cv=cv)
print(scores.mean())

In [ ]:
clf.fit(X_train,Y_train)

In [ ]:
prediction=clf.predict(X_test)
mapping={0:1,1:2,2:3,3:4,4:5,5:6,6:7}
prediction=[mapping[y] for y in predictions]
data_submission=pd.DataFrame()
data_submission['Id']=test_id
data_submission["Cover_Type"]=prediction
data_submission.to_csv("my_submission.csv",index=False)